In [34]:
# Kaggle Comprtition URL
# https://www.kaggle.com/competitions/nlp-getting-started/overview
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from scipy.sparse import hstack
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import csr_matrix, hstack
dir_path = './data/'

In [35]:
# Load train/test data
train_data = pd.read_csv(dir_path + 'train.csv')
test_data = pd.read_csv(dir_path + 'test.csv')

In [36]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [37]:
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [38]:
#see if the text is positive or negative
def get_sentiment(text):
  analysis = TextBlob(text)
  return analysis.sentiment.polarity

In [39]:
import re
def clean(text):
  text = re.sub(r'https?://\S+|www\.\S+', '', text) #remove URLs
  text = re.sub(r'\@|\#|\d+|[|]_-', '', text)
  text = re.sub('<.*?>', '', text) #remove HTML
  return text

In [40]:
train_data['clean_text']=train_data['text'].apply(clean)
x_train = train_data['clean_text']
y_train = train_data['target']

#get the sentimet of the train text
#train_data['sentiment'] = train_data['text'].apply(get_sentiment) #no change
#test_data['sentiment'] = test_data['text'].apply(get_sentiment)

#get the length of both text
#train_data["text_length"] = train_data["text"].apply(len) #worse
#test_data["text_length"] = test_data["text"].apply(len)

test_data['clean_text']=test_data['text'].apply(clean)
x_test = test_data['clean_text']

train_data['text_length'] = train_data['text'].apply(len)
train_data['word_count'] = train_data['text'].apply(lambda x: len(x.split()))
test_data['text_length'] = test_data['text'].apply(len)
test_data['word_count'] = test_data['text'].apply(lambda x: len(x.split()))

In [41]:
#ignore terms that are too frequent, remove stop words, and have ngrams
count_vectorizer = CountVectorizer(max_df=0.5,ngram_range=(1,3),stop_words='english')

x_train_vectors = count_vectorizer.fit_transform(x_train)
x_test_vectors = count_vectorizer.transform(x_test)

# Step 2: Encode 'keyword' using LabelEncoder (convert categorical to numeric)
ohe = OneHotEncoder(sparse_output=False)
train_key = ohe.fit_transform(train_data[['keyword']])
test_key = ohe.transform(test_data[['keyword']])

numeric_features_train = csr_matrix(train_data[['text_length', 'word_count']].values)
numeric_features_test = csr_matrix(test_data[['text_length', 'word_count']].values)

#combin all vectors into matrix
x_train_vectors = hstack([train_key,x_train_vectors,numeric_features_train])  #don't need this if don't use previous two
x_test_vectors = hstack([test_key,x_test_vectors,numeric_features_test])

In [42]:
# see if there is difference between TfidfVectorizer and CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.5,ngram_range=(1,3),stop_words='english')

x_tfidf_train_vectors = tfidf_vectorizer.fit_transform(x_train)
x_tfidf_test_vectors = tfidf_vectorizer.transform(x_test)

## RidgeClassifier

In [43]:
# train the classifier to address overfitting and make predictions
rclf = RidgeClassifier()
rclf.fit(x_train_vectors,y_train)

rclf_prediction = rclf.predict(x_test_vectors)

# submission_data = pd.DataFrame({'id':test_data['id'], 'target':rclf_prediction})

# submission_data.to_csv('RidgeClassifier_alpha7_submission.csv',index=False)

Public score: 0.79344

In [44]:
rclf = RidgeClassifier(tol=0.01)
rclf.fit(x_tfidf_train_vectors,y_train)

rclf_tfidf_prediction = rclf.predict(x_tfidf_test_vectors)

#submission_data = pd.DataFrame({'id':test_data['id'], 'target':rclf_tfidf_prediction})

#submission_data.to_csv('RidgeClassifier_tfidf_tol0.01_submission.csv',index=False)

Public score: 0.80324

## LogisticRegression

In [45]:
from sklearn import linear_model

logr = linear_model.LogisticRegression()
logr.fit(x_train_vectors,y_train)
logr_prediction = logr.predict(x_test_vectors)

c:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [46]:
# submission_data = pd.DataFrame({'id':test_data['id'], 'target':logr_prediction})

# submission_data.to_csv('LogisticRegression_submission.csv',index=False)

In [47]:
#testing different ways of regularization in LogisticRegression (reduce error and overfitting)
l1_model = linear_model.LogisticRegression(penalty='l1', solver='liblinear', C=1.0, random_state=42, max_iter=1000)
l2_model = linear_model.LogisticRegression(penalty='l2', solver='lbfgs', C=1.0, random_state=42, max_iter=1000)  #defult setting max_iter=1000 making it perform better than original
#elastic_net_model = linear_model.LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, C=1.0, random_state=42)

# Train the model
l1_model.fit(x_train_vectors,y_train)
l2_model.fit(x_train_vectors,y_train)
#elastic_net_model.fit(x_train_vectors,y_train)

l1_model_pre = l1_model.predict(x_test_vectors)
l2_model_pre = l2_model.predict(x_test_vectors)
#elastic_net_model_pre = elastic_net_model.predict(x_test_vectors)

# submission_data = pd.DataFrame({'id':test_data['id'], 'target':l1_model_pre})
# submission_data.to_csv('LogrL1_submission.csv',index=False)

# submission_data = pd.DataFrame({'id':test_data['id'], 'target':l2_model_pre})
# submission_data.to_csv('LogrL2_submission.csv',index=False)

#submission_data = pd.DataFrame({'id':test_data['id'], 'target':elastic_net_model_pre})
#submission_data.to_csv('Logr_elastic_submission.csv',index=False)



1. L1 Public score: 0.79313
2. L2 Public score: 0.80110
3. Elastic_net_model Public score: 0.79865
4. L1_tfidf Public score: 0.76739
5. L2_tfidf Public score: 0.80110

check accuracy

In [48]:
#Check the accuracy score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

y = train_data['target']

# Split the data into training and testing sets
# y_test is the true labels and y_pred are the model's predictions
X_train, X_val, y_train, y_test = train_test_split(x_tfidf_train_vectors, y, test_size=0.2, random_state=42)


# Initialize and train the classifier
#rclf = linear_model.LogisticRegression(penalty='l2', solver='lbfgs', C=1.0, random_state=42, max_iter=1000)
rclf.fit(X_train, y_train)

# our prediction
y_pred = rclf.predict(X_val)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Precision: 0.8796498905908097
Recall: 0.6194144838212635
F1 Score: 0.7269439421338157


Precision: 0.8217054263565892
Recall: 0.6533127889060092
F1 Score: 0.7278969957081545

## VotingClassifier

In [51]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

Xtrain, Xtest, ytrain, ytest = train_test_split(x_train_vectors,y, test_size=0.33, random_state=42)

In [52]:
# 定義多個模型
# param_grid = {
#     'max_depth': [3, 5, 10, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, scoring='f1', cv=5)
# grid_search.fit(Xtrain, ytrain)

# best_tree_clf = grid_search.best_estimator_
log_reg = LogisticRegression(penalty='l2', solver='lbfgs', C=1.0, random_state=42, max_iter=1000,class_weight='balanced')
rg_clf = RidgeClassifier(alpha= 7, tol= 0.0001,class_weight='balanced')
xgb_clf = XGBClassifier(
    objective='binary:logistic',  # 二元分類任務
    eval_metric='logloss',       # 評估指標，可以換成 'auc' 或 'error'
    use_label_encoder=False,     # 避免警告
    n_estimators=500,            # 樹的數量
    learning_rate=0.3,           # 學習率
    max_depth=5,                 # 樹的最大深度
    random_state=42
)
rf_clf = RandomForestClassifier(
    n_estimators=100,           # Number of trees in the forest
    max_depth=10,               # Max depth of each tree
    random_state=42,
    class_weight='balanced'     # Optional: Balance the classes in the dataset
)
#tree_clf = DecisionTreeClassifier(criterion='entropy',max_depth=100,min_samples_leaf=1,min_samples_split=50,random_state=42)


# 投票分類器
voting_clf = VotingClassifier(
    estimators=[('log_reg', log_reg), ('rg', rg_clf),('xgb',xgb_clf),('rf',rf_clf)], #, ('tree', best_tree_clf)
    voting='hard',
)

# 訓練與評估
voting_clf.fit(Xtrain, ytrain)
y_pred = voting_clf.predict(Xtest)

print(f"F1分數: {f1_score(ytest, y_pred)}")
print("voting:",classification_report(ytest,y_pred))

# voting_clf.fit(x_train_vectors, y)
# y_pre = voting_clf.predict(x_test_vectors)
# submission_data = pd.DataFrame({'id':test_data['id'], 'target':y_pre})
# submission_data.to_csv('voting_submission.csv',index=False)

c:\anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:25:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


F1分數: 0.7351063829787235
voting:               precision    recall  f1-score   support

           0       0.78      0.92      0.84      1446
           1       0.85      0.65      0.74      1067

    accuracy                           0.80      2513
   macro avg       0.81      0.78      0.79      2513
weighted avg       0.81      0.80      0.80      2513



public score:0.80171   (舊程式分數有比較高0.80447 但我找不回來)